In [5]:
import requests
import json
import os



Error: 429 {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}



In [8]:
# Select a single email (first row for testing purposes)
email_row = enhanced_classified_emails.iloc[6]

# Extract the topic from the email
topic = email_row['topic']

# Fetch corresponding stats for the topic from the topic summary
stats = topic_summary.loc[topic_summary['topic'] == topic]

# Check if stats exist for the given topic
if stats.empty:
    print(f"No stats found for topic: {topic}")
else:
    # Extract statistics as a dictionary for prompt generation
    stats_dict = {
        'topic': topic,
        'average_word_count': stats['avg_word_count'].values[6],
        'std_word_count': stats['std_word_count'].values[6],
        'average_body_length': stats['avg_body_length'].values[6],
        'std_body_length': stats['std_body_length'].values[6],
        'average_hour': stats['avg_hour'].values[6],
        'std_hour': stats['std_hour'].values[6],
        'formality_average': stats['avg_formality_ratio'].values[6],
        'formality_std': stats['std_formality_ratio'].values[6]
    }

    # Generate a GPT prompt based on the single email and topic stats
    prompt = (
        f"Write an email similar to the following example:\n\n"
        f"Subject: {email_row['subject']}\n\n"
        f"Here are the average characteristics of emails for the topic '{stats_dict['topic']}':\n"
        f"- Average word count: {stats_dict['average_word_count']} (±{stats_dict['std_word_count']})\n"
        f"- Average body length: {stats_dict['average_body_length']} (±{stats_dict['std_body_length']})\n"
        f"- Average hour sent: {stats_dict['average_hour']} (±{stats_dict['std_hour']})\n"
        f"- Formality score: {stats_dict['formality_average']} (±{stats_dict['formality_std']})\n\n"
        f"Use a semi-formal tone. The email should match the given characteristics."
    )

    # Display the generated prompt
    prompt


In [ ]:
import pandas as pd
import requests


# 1. Load your data
enhanced_classified_emails = pd.read_csv("enhanced_classified_emails.csv")
topic_summary = pd.read_excel("topic_summary_no_outliers_IQR.xlsx")

# 2. Define a helper function to call ChatGPT API
def call_chatgpt_api(prompt, api_key):
    """
    Send a single prompt to the ChatGPT API and return the response text.
    Each call is effectively a new 'session' unless you pass conversation history.
    """
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": "gpt-4",  # or 'gpt-3.5-turbo' if you prefer
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are writing emails using statistics (but do not include them in the mail). "
                    "Do not give numbers or direct data, but write an email regarding the subject. "
                    "Use a semi-formal tone. "
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print("Error:", response.status_code, response.text)
        return None

# 3. Prepare to store results
all_results = []

# 4. Choose a single row index to process (e.g., index 5)
idx_to_test = 1

# 5. Fetch the row
email_row = enhanced_classified_emails.iloc[idx_to_test]
topic = email_row['topic']
subject_text = str(email_row.get('subject', ''))
body_text = str(email_row.get('body', ''))

# Default placeholders
prompt = ""
gpt_response = None

# 6. Check the skip conditions
if topic == "Empty":
    # If topic is "Empty," skip generating an email
    prompt = "Skipped because topic is 'Empty'."
elif "QXJkYSBB" in subject_text or "QXJkYSBB" in body_text:
    # Skip if the forbidden string is in the subject or body
    prompt = "Skipped because 'QXJkYSBB' substring was found in subject or body."
else:
    # 7. Fetch stats for this topic from topic_summary
    stats = topic_summary.loc[topic_summary['topic'] == topic]
    
    if stats.empty:
        # No stats found, skip the API
        prompt = f"No stats found for topic: {topic}"
        gpt_response = None
    else:
        # Build the stats_dict (ensure column names match your topic_summary)
        stats_dict = {
            'topic': topic,
            'average_word_count':       stats['avg_word_count'].values[0],
            'std_word_count':           stats['std_word_count'].values[0],
            'average_body_length':      stats['avg_body_length'].values[0],
            'std_body_length':          stats['std_body_length'].values[0],
            'average_hour':             stats['avg_hour'].values[0],
            'std_hour':                 stats['std_hour'].values[0],
            'formality_average':        stats['avg_formality_ratio'].values[0],
            'formality_std':            stats['std_formality_ratio'].values[0]
        }

        # 8. Generate the GPT prompt
        prompt = (
            f"Write an email similar to the following example:\n\n"
            f"Subject: {subject_text}\n\n"
            f"Here are the average characteristics of emails for the topic '{stats_dict['topic']}':\n"
            f"- Average word count: {stats_dict['average_word_count']} (±{stats_dict['std_word_count']})\n"
            f"- Average body length: {stats_dict['average_body_length']} (±{stats_dict['std_body_length']})\n"
            f"- Average hour sent: {stats_dict['average_hour']} (±{stats_dict['std_hour']})\n"
            f"- Formality score: {stats_dict['formality_average']} (±{stats_dict['formality_std']})\n\n"
            f"Do NOT use these statistics or direct numbers in the final email. "
            f"Write it as if it's real life."
        )

        # 9. Call the ChatGPT API
        openai_api_key = "sk-***********zD5YPT_xXhHRXXiHtyy4iMZgywbT3BlbkFJxnLBjxWDh22cjl-ISWIvhD1HEBOTWwR5neJFfpsj8A"  # This key is deactivated
        gpt_response = call_chatgpt_api(prompt, openai_api_key)

# 10. Append results for this single row
all_results.append({
    'row_index': idx_to_test,
    'topic': topic,
    'subject': subject_text,
    'original_body': body_text,
    'prompt_used': prompt,
    'gpt_generated_body': gpt_response
})

# 11. Convert all_results to a DataFrame and save to Excel
results_df = pd.DataFrame(all_results)
results_df.to_excel("gpt_output_single.xlsx", index=False)

print(f"Done! Processed row index {idx_to_test}. Saved the result to 'gpt_output_single.xlsx'.")


Done! Processed row index 1. Saved the result to 'gpt_output_single.xlsx'.


In [55]:
import pandas as pd
import requests

# 1. Load your data
enhanced_classified_emails = pd.read_csv("enhanced_classified_emails.csv")
topic_summary = pd.read_excel("topic_summary_no_outliers_IQR.xlsx")

# 2. Define a helper function to call ChatGPT API
def call_chatgpt_api(prompt, api_key):
    """
    Send a single prompt to the ChatGPT API and return the response text.
    Each call is effectively a new 'session' unless you pass conversation history.
    """
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": "gpt-4",  # or 'gpt-3.5-turbo' if you prefer
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are writing emails using statistics (but do not include them in the mail). "
                    "Do not give numbers or direct data, but write an email regarding the subject. "
                    "Use a semi-formal tone."
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print("Error:", response.status_code, response.text)
        return None

# 3. Prepare to store results
all_results = []

# 4. Iterate over all rows in enhanced_classified_emails
for idx in range(len(enhanced_classified_emails)):
    email_row = enhanced_classified_emails.iloc[idx]
    topic = email_row['topic']
    subject_text = str(email_row.get('subject', ''))
    body_text = str(email_row.get('body', ''))

    prompt = ""
    gpt_response = None

    # A) Skip if topic == "Empty"
    if topic == "Empty":
        prompt = "Skipped because topic is 'Empty'."
        # gpt_response remains None
    # B) Skip if "QXJkYSBB" found in subject or body
    elif "QXJkYSBB" in subject_text or "QXJkYSBB" in body_text:
        prompt = "Skipped because 'QXJkYSBB' substring was found in subject or body."
    else:
        # C) Otherwise, fetch stats for this topic
        stats = topic_summary.loc[topic_summary['topic'] == topic]

        if stats.empty:
            prompt = f"No stats found for topic: {topic}"
        else:
            # Build the stats_dict (ensure these column names match your topic_summary)
            stats_dict = {
                'topic': topic,
                'average_word_count':       stats['avg_word_count'].values[0],
                'std_word_count':           stats['std_word_count'].values[0],
                'average_body_length':      stats['avg_body_length'].values[0],
                'std_body_length':          stats['std_body_length'].values[0],
                'average_hour':             stats['avg_hour'].values[0],
                'std_hour':                 stats['std_hour'].values[0],
                'formality_average':        stats['avg_formality_ratio'].values[0],
                'formality_std':            stats['std_formality_ratio'].values[0]
            }

            # Generate the GPT prompt
            prompt = (
                f"Write an email similar to the following example:\n\n"
                f"Subject: {subject_text}\n\n"
                f"Here are the average characteristics of emails for the topic '{stats_dict['topic']}':\n"
                f"- Average word count: {stats_dict['average_word_count']} (±{stats_dict['std_word_count']})\n"
                f"- Average body length: {stats_dict['average_body_length']} (±{stats_dict['std_body_length']})\n"
                f"- Average hour sent: {stats_dict['average_hour']} (±{stats_dict['std_hour']})\n"
                f"- Formality score: {stats_dict['formality_average']} (±{stats_dict['formality_std']})\n\n"
                f"Do NOT use these statistics or direct numbers in the final email. "
                f"Write it as if it's real life."
            )

            # Call the ChatGPT API
            openai_api_key = "sk-***********zD5YPT_xXhHRXXiHtyy4iMZgywbT3BlbkFJxnLBjxWDh22cjl-ISWIvhD1HEBOTWwR5neJFfpsj8A"  # This key is deactivated
            gpt_response = call_chatgpt_api(prompt, openai_api_key)

    # Save the results for this row
    all_results.append({
        'row_index': idx,
        'topic': topic,
        'subject': subject_text,
        'original_body': body_text,
        'prompt_used': prompt,
        'gpt_generated_body': gpt_response
    })

# 5. Convert all_results to a DataFrame and save to Excel
results_df = pd.DataFrame(all_results)
results_df.to_excel("gpt_output_all.xlsx", index=False)

print("Done! Processed all rows in 'enhanced_classified_emails.csv' and saved the results to 'gpt_output_all.xlsx'.")


Done! Processed all rows in 'enhanced_classified_emails.csv' and saved the results to 'gpt_output_all.xlsx'.


In [56]:
# 5. Convert all_results to a DataFrame and save to Excel
results_df = pd.DataFrame(all_results)
results_df.to_excel("gpt_output_all.xlsx", index=False)

print("Done! Processed all rows in 'enhanced_classified_emails.csv' and saved the results to 'gpt_output_all.xlsx'.")

Done! Processed all rows in 'enhanced_classified_emails.csv' and saved the results to 'gpt_output_all.xlsx'.
